In [23]:
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np
from sklearn import metrics
import import_ipynb
import Callbacks

In [21]:
def build_model(inputShape=(128, 128, 3), numClasses=3):

    model = models.Sequential()

    # Convolutional and Pooling Layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=inputShape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the feature map and add Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout to prevent overfitting
    model.add(layers.Dense(numClasses, activation='softmax'))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

    return model

In [22]:
def trainModel(model, train, validation, epochs, device):
    # Initialize Callbacks
    earlyStoppingF1 = Callbacks.EarlyStoppingByF1(patience=5, monitor='val_f1_score')
    sklearnMetricsCallback = Callbacks.SklearnMetricsCallback(validationData=validation, earlyStoppingMonitor=earlyStoppingF1)

    if device=="GPU":
        print("Training on GPU\n")
        with tf.device("/GPU:0"):
            history = model.fit(train, epochs=epochs, validation_data=validation, callbacks=[sklearnMetricsCallback, earlyStoppingF1])
    else:
        print("Training on CPU\n")
        with tf.device("/CPU:0"):
            history = model.fit(train, epochs=epochs, validation_data=validation, callbacks=[sklearnMetricsCallback, earlyStoppingF1])
            
    return history

In [ ]:
def plotModelHistory(history):
    pass

In [25]:
def evaluateModel(model, test):
    y_pred = model.predict(test, verbose=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    y_true = test.classes

    class_names = test.class_indices
    class_names = {v: k for k, v in class_names.items()}
    
    report = metrics.classification_report(y_true, y_pred_classes, target_names=list(class_names.values()))

    return report